source：https://github.com/AlexLynd/ESP8266-Wardriving

the dataset: gathered the data by walking around with WiGLE, exported to csv

the task: filter data by different criteria, plot on the map

In [ ]:
import numpy as np
import pandas as pd
import folium
from folium import Map, Marker
import random
import matplotlib.pyplot as plt
color_list = ['black', 'darkblue', 'cadetblue', 'pink', 'darkred', 'white', 'orange', 'purple', 'lightblue', 'darkpurple', 'gray', 'red', 'lightgray', 'lightred', 'green', 'darkgreen', 'beige', 'lightgreen', 'blue']
from tqdm import tqdm
import time

In [ ]:
def importdataset():
    global wd
    wd = pd.read_csv ('WigleWifi_20210906181508.csv', skiprows=1) 
    print(f"total devices: {len(wd)}") # number of detected devices

# explore and clean the data

In [ ]:
importdataset()
wd.sample(5)

In [ ]:
wd["MAC"].value_counts()

In [ ]:
#take a look how many unique valid MACs are there and how many entries in total
wd.loc[(wd['MAC'].str.len() == 17)]["MAC"].describe()

In [ ]:
print(f"number of unique APs: {len(wd['MAC'].unique())}")

In [ ]:
pd.DataFrame (wd['MAC'].unique())

In [ ]:
pd.DataFrame (wd.loc[(wd['MAC'].str.len() == 17)]["MAC"])

# plot all WCDMA towers

In [ ]:
importdataset()
wd.sample(5)

In [ ]:
# drop all else except WCDMA
wd = pd.DataFrame( wd.loc[wd['Type'] == "WCDMA"] )
wd = wd.reset_index(drop=True)
wd.sample(5)

In [ ]:
wd['MAC'].value_counts()

In [ ]:
#color code different MACs
wd["Color"] = None
for index, row in wd.iterrows():
    if wd['Color'][index] == None:
        wd.loc[wd['MAC'] == wd['MAC'][index],"Color"] = random.choice(color_list)

In [ ]:
pd.DataFrame (wd, columns=['Color','MAC']).value_counts()

In [ ]:
lat = wd["CurrentLatitude"][0]
lon = wd["CurrentLongitude"][0]
basic_map = folium.Map(location=[lat, lon],zoom_start=17) # manually pick scope of map

# folium.PolyLine(wd[['CurrentLatitude', 'CurrentLongitude']].values.tolist(),line_opacity = 0.5, weight = 4).add_to(basic_map) # plot route

# add markers for every point in dataset 
for i in range(len(wd)): 
#     if wd["FirstSeen"][i] > "2021-8-21 18:43:25" and wd["FirstSeen"][i] < "2021-8-21 18:46:58": # add time filter
#     if wd["RSSI"][i] > 10:
    if wd.loc[wd['MAC'] == wd['MAC'][i],"RSSI"].max() == wd["RSSI"][i]:# only plot with highest signal strength
        lat = wd["CurrentLatitude"][i]
        lon = wd["CurrentLongitude"][i] 
        folium.Marker( location=[lat, lon], tooltip=f'MAC {wd["MAC"][i]}, strength {wd["RSSI"][i]}', icon=folium.Icon(icon="wifi", color=wd["Color"][i], prefix='fa') ).add_to(basic_map) # annotate GPS location w/ SSID
print(f"total towers: {len(wd['MAC'].unique())}")        
display(basic_map)

# 

# 

# 

# plot all WIFI APs with highest signal strength

In [ ]:
importdataset()
wd.sample(5)

In [ ]:
#drop everything except those having valid MACs
wd = wd.loc[(wd['MAC'].str.len() == 17)]
wd = wd.reset_index(drop=True)
wd.sample(5)

In [ ]:
wd.info()

In [ ]:
print(f"total APs: {len(wd['MAC'].unique())}") 

In [ ]:
lat = wd["CurrentLatitude"][0]
lon = wd["CurrentLongitude"][0]
basic_map = folium.Map(location=[lat, lon],zoom_start=17) # manually pick scope of map

# add markers for every point in dataset 
for i in range(len(wd)): 
    if wd.loc[wd['SSID'] == wd['SSID'][i],"RSSI"].max() == wd["RSSI"][i]: # only plot AP with highest signal strength
        lat = wd["CurrentLatitude"][i]
        lon = wd["CurrentLongitude"][i]
        folium.Marker( location=[lat, lon], tooltip=f'SSID {wd["SSID"][i]}, strength {wd["RSSI"][i]}', icon=folium.Icon(icon="wifi", color="blue", prefix='fa') ).add_to(basic_map) # annotate GPS location w/ SSID
print(f"total APs: {len(wd['MAC'].unique())}")    
display(basic_map)

# 

# 

# 

# 

# plot all WPS-enabled wifi with highest signal strength

In [ ]:
importdataset()
wd.sample(5)

In [ ]:
wd['AuthMode'].value_counts()

In [ ]:
# make a sorted list of authmodes used
wd_sorted_list = pd.DataFrame (wd, columns=['AuthMode']).value_counts().rename_axis('Auth_sorted').reset_index(name='counts')
wd_sorted_list

In [ ]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = wd_sorted_list['Auth_sorted']
sizes = wd_sorted_list['counts']

fig1, ax1 = plt.subplots()
ax1.pie(sizes,  labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

In [ ]:
ap_total=len(wd)
ap_use_wps=len(pd.DataFrame( wd.loc[lambda x: (wd['AuthMode'].str.find('WPS') != -1), :]))
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = "all APs", "APs that use WPS"
sizes = [ap_total, ap_use_wps]

fig1, ax1 = plt.subplots()
ax1.pie(sizes,  labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

In [ ]:
# drop all except those that use WPS
wd = pd.DataFrame( wd.loc[lambda x: (wd['AuthMode'].str.find('WPS') != -1), :])
wd = wd.reset_index(drop=True)
wd.sample(5)

In [ ]:
wd['AuthMode'].value_counts()

In [ ]:
# make a sorted list of authmodes used
wd_sorted_list = pd.DataFrame (wd, columns=['AuthMode']).value_counts().rename_axis('Auth_sorted').reset_index(name='counts')
wd_sorted_list

In [ ]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = wd_sorted_list['Auth_sorted']
sizes = wd_sorted_list['counts']

fig1, ax1 = plt.subplots()
ax1.pie(sizes,  labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

In [ ]:
#color code authmodes
wd["Color"] = None
for index, row in wd.iterrows():
    if wd['Color'][index] == None:
        wd.loc[wd['AuthMode'] == wd['AuthMode'][index],"Color"] = random.choice(color_list)

In [ ]:
pd.DataFrame (wd, columns=['Color','AuthMode']).value_counts()

In [ ]:
lat = wd["CurrentLatitude"][0]
lon = wd["CurrentLongitude"][0]
basic_map = folium.Map(location=[lat, lon],zoom_start=17) # manually pick scope of map

# add markers for every point in dataset 
for i in range(len(wd)): 

    if wd.loc[wd['SSID'] == wd['SSID'][i],"RSSI"].max() == wd["RSSI"][i]: # only plot AP with highest signal strength
        lat = wd["CurrentLatitude"][i]
        lon = wd["CurrentLongitude"][i]
        folium.Marker( location=[lat, lon], tooltip=f'SSID {wd["SSID"][i]}, strength {wd["RSSI"][i]}', icon=folium.Icon(icon="wifi", color=wd["Color"][i], prefix='fa') ).add_to(basic_map) # annotate GPS location w/ SSID
print(f"total APs: {len(wd['MAC'].unique())}")    
display(basic_map)

# add vendor names to the dataset

In [ ]:
# https://github.com/AlexLynd/ESP8266-Wardriving/blob/master/mac-vendors.txt
mac_vendors = pd.read_csv ('mac-vendors.csv', names=("MAC", "Vendor")) 
mac_vendors["MAC"]= mac_vendors["MAC"].str.lower()
mac_vendors

In [ ]:
importdataset()
wd["MAC"]= wd["MAC"].str.lower()

In [ ]:
#drop everything except those having valid MACs
wd = wd.loc[(wd['MAC'].str.len() == 17)]
wd = wd.drop_duplicates(subset = ["MAC"])
wd = wd.reset_index(drop=True)
len(wd)

In [ ]:
mac_vendors.loc[mac_vendors['MAC'] == wd['MAC'][0][:8],"Vendor"]

In [ ]:
mac_vendors.loc[mac_vendors['MAC'] == wd['MAC'][0][:8],"Vendor"].values[0]

In [ ]:
# wd['Vendor'] = None
# for index, row in wd.iterrows():
#     if wd['Vendor'][index] == None:
#         cur_data = mac_vendors.loc[mac_vendors['MAC'] == row['MAC'][:8],"Vendor"]
#         if len(cur_data) >0: #if there's a match
#             wd["Vendor"][index] = cur_data.values[0]
            


In [ ]:
wd['Vendor'] = None
iters=0
filled=0
start_time = time.time()
with tqdm(total=len(wd)) as bar:
    for index, row in wd.iterrows():
        bar.update(1)
        if wd['Vendor'][index] == None:
            iters+=1
            cur_data = mac_vendors.loc[mac_vendors['MAC'] == row['MAC'][:8],"Vendor"]
            if len(cur_data) >0: #if there's a match
                wd.loc[row['MAC'][:8] == mac_vendors['MAC'],"Vendor"] = cur_data.values[0]
                filled+=1
print(f"it took  {time.time()-start_time:.2f} sec. to run the code")           
print(f"total rows: {len(wd)}, total iterations: {iters}, filled cells: {filled}")

In [ ]:
wd['Vendor'].isna().sum()

In [ ]:
len(wd)

In [ ]:
print(f"number of unique MACs: {len(mac_vendors['MAC'].unique())}")

In [ ]:
len(mac_vendors)

In [ ]:
wd["Vendor"].value_counts().head(30)

In [ ]:
# 